<a href="https://colab.research.google.com/github/Atomnp/realtime_text_similarity_backend/blob/main/minor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from typing import List
from sklearn.feature_extraction.text import (
    TfidfVectorizer,
    CountVectorizer,
    TfidfTransformer,
)
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix, lil_matrix
import numpy as np
import itertools
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# mounting your google drive to colab
from google.colab import drive

drive.mount("/gdrive")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


**Make shortcut of [this](https://drive.google.com/drive/folders/1BGr0cWKiJwT_jNg9nRNAhWgy0mYPgw_K?usp=sharing) folder in your gdrive**

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec


class callback(CallbackAny2Vec):
    """Callback to print loss after each epoch."""

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print("Loss after epoch {}: {}".format(self.epoch, loss))
        else:
            print(
                "Loss after epoch {}: {}".format(
                    self.epoch, loss - self.loss_previous_step
                )
            )
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# load dataset
dataset = r"/gdrive/MyDrive/minor_project_files/questions.csv"

df = pd.read_csv(dataset, keep_default_na=False, na_values=["_"])
questions = df.loc[:, "question1"].to_list()

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
# train and save model
sentences = (word_tokenize(sentence) for sentence in questions)

# model = Word2Vec(sentences=list(sentences), size=100, window=5, min_count=1, workers=4, compute_loss=True, iter=6, callbacks=[callback()])
# model.save("/gdrive/MyDrive/minor_project_files/word2vec.model")

In [ ]:
def word2vec(word):
    model = Word2Vec.load("/gdrive/MyDrive/minor_project_files/word2vec.model")
    return model.wv[word]

In [ ]:
# word_vec= word2vec('king')
# print(word_vec)
model = Word2Vec.load("/gdrive/MyDrive/minor_project_files/word2vec.model")
# print(model.wv.most_similar(positive=['king'], negative=[], topn=50))

In [ ]:
# not used
idf_scores = {}
key_words = []
tfidf_matrix = []

In [ ]:
# not used
def extract_key_words(corpus: List[List[str]], num_of_words=3):
    global idf_scores
    global key_words
    global tfidf_matrix

    def identity_tokenizer(text):
        return text

    # vect = TfidfVectorizer(max_df=0.8, max_features=200000, min_df=0.2, stop_words=nltk.corpus.stopwords.words('english'), use_idf=True, tokenizer=identity_tokenizer,lowercase=False)
    vect = TfidfVectorizer(
        stop_words=nltk.corpus.stopwords.words("english"),
        use_idf=True,
        tokenizer=identity_tokenizer,
        lowercase=True,
    )
    tfidf_matrix = vect.fit_transform(corpus)

    print(vect.idf_)

    idf_scores = dict(zip(vect.get_feature_names(), vect.idf_))

    test = vect.fit(["this is some random text i got from somewhere".split()])
    print(test)
    print(vect.get_feature_names())

    # df = pd.DataFrame(tfidf_matrix.toarray(), columns=vect.get_feature_names())
    # key_words = df.apply(lambda x:x.nlargest(num_of_words).index.values,axis=1)

    # return df.apply(lambda x:x.nlargest(num_of_words).index.values,axis=1).to_list()

In [ ]:
# extract_key_words(itertools.islice(sentences, 50000))
# extract_key_words(sentences)

^^ First train the model on the entire dataset

In [ ]:
def identity_tokenizer(text):
    return text


# vect = TfidfVectorizer(max_df=0.8, max_features=200000, min_df=0.2, stop_words=nltk.corpus.stopwords.words('english'), use_idf=True, tokenizer=identity_tokenizer,lowercase=False)
vect = TfidfVectorizer(
    stop_words=nltk.corpus.stopwords.words("english"),
    use_idf=True,
    tokenizer=identity_tokenizer,
    lowercase=False,
)
# copy the iterator so that the cell can be rerun (otherwise the iterator will be at the end)
it_copy, sentences = itertools.tee(sentences)
tfidf_matrix = vect.fit_transform(it_copy)
fv = vect.get_feature_names()

# print(vect.idf_, len(vect.idf_))

CPU times: user 6.03 s, sys: 142 ms, total: 6.17 s
Wall time: 6.52 s


In [ ]:
# implementation to find sentence embeddings (alternative 1) : lil_matrix! 2m4s!!
it_copy, sentences = itertools.tee(sentences)
sem = []
# cx = coo_matrix(tfidf_matrix)
cx = lil_matrix(tfidf_matrix)

In [ ]:
sem = []
to = cx.get_shape()[0]
for i in range(to):
    rx = cx.getrow(i).tocoo()

    sorted_by_tfidf = sorted(
        [(fv[j], v) for k, j, v in zip(rx.row, rx.col, rx.data)],
        key=lambda x: x[1],
        reverse=True,
    )
    arrlist = np.array(list(map(lambda x: model.wv[x[0]], sorted_by_tfidf[:5])))
    sem.append(np.mean(arrlist, axis=0))

sem = np.asarray(sem)
np.save("/gdrive/MyDrive/minor_project_files/sentence_embeddings2.npy", sem)

In [ ]:
# implementation to find sentence embeddings (alternative 2) faster: 13 minutes

# it_copy, sentences = itertools.tee(sentences)
# sem = []
# for sentence in itertools.islice(it_copy, 1000):
#     # print(sentence)
#     matrix = vect.transform([sentence])
#     # iterating over the sparse matrix representatin using COOrdinate matrix
#     cx = coo_matrix(matrix)

#     sorted_by_tfidf = sorted([(fv[j],v) for i,j,v in zip(cx.row, cx.col, cx.data)], key=lambda x: x[1], reverse=True)
#     arrlist = np.array( list(map(lambda x: model.wv[x[0]], sorted_by_tfidf[:5])  ))
#     sem.append(np.mean(arrlist, axis=0))
# sem = np.asarray(sem)
# np.save('/gdrive/MyDrive/minor_project_files/sentence_embeddings.npy', sem)

In [ ]:
# save idf_scores (but we dont need to do this it seems)
# idf_scores = dict(zip(vect.get_feature_names(), vect.idf_))

In [ ]:
# sample block on how to get the tfidf score of new question
# get similar words from question string
def get_similar_words(question: str):
    to_transform = [question.split()]
    matrix = vect.transform(to_transform)
    print(matrix)

    # iterating over the sparse matrix representatin using COOrdinate matrix
    cx = coo_matrix(matrix)
    sorted_by_tfidf = sorted(
        [(fv[j], v) for i, j, v in zip(cx.row, cx.col, cx.data)],
        key=lambda x: x[1],
        reverse=True,
    )
    print(sorted_by_tfidf)

    most_similar = []
    for i in range(len(sorted_by_tfidf)):
        element = sorted_by_tfidf[i]
        top3similar = model.wv.most_similar(positive=[element[0]], negative=[], topn=3)
        most_similar += [(k[0], k[1] * element[1]) for k in top3similar]

    most_similar = sorted(most_similar, key=lambda x: x[1], reverse=True)
    print(most_similar)
    return most_similar

In [ ]:
print(get_similar_words("what is the best way to invest in India stock market?"))

^^ 

In [ ]:
def similarity_two_key_words(v1, v2):
    # s_k=extract_key_words([corpus_s],5)[0]
    # print(v1,s_k)
    ev1 = [model.wv[word] for word in v1]
    ev2 = [model.wv[word] for word in v2]
    sim = cosine_similarity(ev1, ev2)
    return np.prod(np.amax(sim, axis=1))

In [ ]:
def get_similar(sentence: str, n: int = 5):
    # print(word_tokenize(sentence))
    # key_words=extract_key_words([word_tokenize(sentence)],5)
    # print(sentences[:1000])

    # corpus=[word_tokenize(sentence)]+sentences[:1000]
    # key_words=extract_key_words(corpus,5)

    score = [similarity_two_key_words(key_words[0], question) for question in key_words]
    similarity = create_dict(score, [sentence] + questions[:1000])

    # sort based on score and return n similar items
    return dict(
        itertools.islice(
            dict(sorted(similarity.items(), key=lambda x: x[0], reverse=True)).items(),
            n,
        )
    )


def create_dict(score, questions):
    return dict(zip(score, questions))

In [ ]:
a = get_similar("who is albert einstein how did he becaome so genius?")
print(a)

Running Flask on collab

In [ ]:
%%capture
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null
!echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
!sudo apt update && sudo apt install ngrok
!pip install flask_ngrok flask-bootstrap
!pip install flask_restful flask_cors
!cat /gdrive/MyDrive/minor_project_files/ngrok_token | xargs ngrok authtoken

In [ ]:
import sys

stdout = sys.stdout
stderr = sys.stderr

In [ ]:
print("test")

test


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request, jsonify
from flask_restful import Resource, Api
import os, logging, sys
from flask_cors import CORS, cross_origin

# sys.stdout = open("/gdrive/MyDrive/minor_project_files/test.txt", "w", buffering=1)
# sys.stderr = open("/gdrive/MyDrive/minor_project_files/test.txt", "a", buffering=1)

app = Flask(__name__)
cors = CORS(app, resources={r"/*": {"origins": "*"}})
# cors = CORS(app)
# app.config['CORS_HEADERS'] = 'Content-Type'
api = Api(app)

run_with_ngrok(app)


class Similarity(Resource):
    # get endpoint to check server is up
    def get(self):
        return jsonify({"hello": "Server Online!"})

    def post(self):
        json_data = request.get_json(force=True)
        qn = json_data["question"]
        # similarity = get_similar(qn)
        # return list of questions
        x = ["question 1", "question 2"]
        return x


api.add_resource(Similarity, "/")
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a520-35-227-155-192.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [02/Feb/2022 15:20:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 15:20:49] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
sys.stdout = stdout
sys.stderr = stderr